In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

# PyTorch CUDA versiyonunu kontrol etme
print("PyTorch CUDA Version:", torch.version.cuda)

# PyTorch'un mevcut CUDA cihazını kontrol etme
print("Current CUDA Device:", torch.cuda.get_device_name(0))

# CUDA sürümünü kontrol etmek için CUDA modülünü yükleme
!nvcc --version

PyTorch CUDA Version: 12.1
Current CUDA Device: Tesla T4
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


Training with params: {'batch_size': 32, 'gamma': 0.1, 'lr': 0.01, 'momentum': 0.8, 'step_size': 10}

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.model_selection import ParameterGrid

# Veri dönüşümleri
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/Deep/chest_xray/chest_xray'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda")

# DenseNet modelini yükleme
def initialize_model(num_classes):
    model_ft = models.densenet121(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    return model_ft

criterion = nn.CrossEntropyLoss()

# Belirtilen parametreler
params = {'batch_size': 32, 'gamma': 0.1, 'lr': 0.01, 'momentum': 0.8, 'step_size': 10}

# DataLoader'ları güncelle
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=params['batch_size'],
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}

model_ft = initialize_model(len(class_names)).to(device)

optimizer_ft = optim.SGD(model_ft.parameters(), lr=params['lr'], momentum=params['momentum'])
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=params['step_size'], gamma=params['gamma'])

def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

def test_model(model, dataloaders):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / dataset_sizes['test']
    return test_acc

# Modeli eğitme ve testi gerçekleştirme
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, num_epochs=15)
test_acc = test_model(model_ft, dataloaders)
print(f"Test Acc: {test_acc:.4f}")

# Modeli kaydetme
best_path = '/content/drive/MyDrive/Deep_Result/DenseNet/Saved_Models\DenseNet_model_with_best_Parameters.pth'
torch.save(model_ft.state_dict(), best_path)
print(f"Model saved to {best_path}")


Epoch 0/14
----------
train Loss: 0.2647 Acc: 0.8931
val Loss: 1.0814 Acc: 0.6875

Epoch 1/14
----------
train Loss: 0.1719 Acc: 0.9299
val Loss: 0.9210 Acc: 0.6250

Epoch 2/14
----------
train Loss: 0.1654 Acc: 0.9422
val Loss: 0.3258 Acc: 0.7500

Epoch 3/14
----------
train Loss: 0.1413 Acc: 0.9446
val Loss: 1.0787 Acc: 0.6250

Epoch 4/14
----------
train Loss: 0.1264 Acc: 0.9523
val Loss: 0.8185 Acc: 0.8125

Epoch 5/14
----------
train Loss: 0.1255 Acc: 0.9559
val Loss: 0.4878 Acc: 0.6875

Epoch 6/14
----------
train Loss: 0.1155 Acc: 0.9579
val Loss: 0.2633 Acc: 0.8750

Epoch 7/14
----------
train Loss: 0.1085 Acc: 0.9600
val Loss: 0.0321 Acc: 1.0000

Epoch 8/14
----------
train Loss: 0.1093 Acc: 0.9550
val Loss: 0.3882 Acc: 0.8125

Epoch 9/14
----------
train Loss: 0.1021 Acc: 0.9653
val Loss: 0.7306 Acc: 0.7500

Epoch 10/14
----------
train Loss: 0.0817 Acc: 0.9692
val Loss: 0.6350 Acc: 0.7500

Epoch 11/14
----------
train Loss: 0.0815 Acc: 0.9728
val Loss: 0.5274 Acc: 0.7500

Ep